**Autor:** Felipe Moreira Sallazar 

**Contato:** sallazarfelipe@gmail.com

**Vídeo de explicação:** [Inteligência Artificial - Aprendizado de Máquina Não Supervisionado](https://youtu.be/OALwnR4HgjU)

**Problema:**

Esta tarefa consiste na execução do algoritmo de aprendizado não supervisionado para agrupamento (k-means) estudado na disciplina, para definição do melhor número de grupos usando o método do “cotovelo”, usando a biblioteca [sklearn](https://scikit-learn.org/stable/index.html). 

 O conjunto de dados pode ser selecionado no repositório [UCI](http://archive.ics.uci.edu/ml/) ou no site da  ferramenta  [Weka](http://www.cs.waikato.ac.nz/ml/weka/)  e  deve  ser  diferente  daquele  usado  nos exemplo da aula e das videoaulas. 


**Exercício:**

As etapas que devem ser executadas e relatadas são:

• Selecionar um conjunto de dados  rotulado, para fazer validação externa usando as  classes; Para o agrupamento, a classe deve ser retirada;

• Ler o conjunto de dados no formato .csv coma função read.csv() (biblioteca Pandas);

• Criar outra tabela de dados retirando a coluna referente às classes;

• Fazer as transformações que forem necessárias: transformar atributos nominais em binários com a função get_dummies() e normalizar os atributos contínuos com a função fit() da classe MinMaxScaler (biblioteca sklearn.preprocessing);

• Aplicar o algoritmo k-means no conjunto de dados sem a classe definindo o número de grupos como o número de classes do conjunto de dados; 

• Aplicar  os  índices  de  validação  ARI  (Adjusted  Rand  Index)  e  AMI  (Adjusted  Mutual  Information) usando as funções disponíveis na biblioteca scikit-learn; 

• Definir  um  intervalo  de  valores  para  número  de  grupos  (por  exemplo,  de  2  a  15,  ou  um  intervalo maior caso seja necessário para visualizar a variação do índice no gráfico); 

• Aplicar o algoritmo k-means no conjunto de dados (sem a classe) para todos os valores do intervalo definido usando a classe KMeans e a função fit() (módulo sklearn.cluster); 

• Coletar o valor do índice soma quadrática das distâncias para todos os agrupamentos encontrados usando o atributo inertia_ da classe KMeans (módulo sklearn.cluster); 

• Coletar o valor do índice de validação silhueta, usando a função da biblioteca scikit-learn; 

• Plotar o gráfico dos índices erro quadrático encontrados para cada número de grupos e determinar qual é o melhor de acordo com o método do cotovelo. 

• Plotar o gráfico dos índices de silhueta encontrados para cada número de grupos e determinar qual é o  melhor de acordo com o método do cotovelo.

**[Link](http://archive.ics.uci.edu/dataset/544/estimation+of+obesity+levels+based+on+eating+habits+and+physical+condition) da base escolhida**

## **Resolução do problema:**

Iniciando com a importação das bibliotecas e da Base de Dados a ser utilizada.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler
from ucimlrepo import fetch_ucirepo
from yellowbrick.cluster import KElbowVisualizer

# fetch dataset 
dataset= fetch_ucirepo(id=544) 

Criação da tabela de dados, matendo os ros targets em uma tabela separada.

In [ ]:
# Data (como pandas dataframes) 
X = dataset.data.features 
Y = dataset.data.targets 
Y.rename(columns={Y.columns[-1]: 'Target'}, inplace=True)

df = pd.concat([X, Y], axis=1)

# Metadata 
print(dataset.metadata) 
# variable information 
print(df.dtypes)
# Data Frame
df

Fazendo a transformações dos atributos nominais em binários e aplicando a normalização dos dados.

In [ ]:
# Transformar atributos nominais em binários (se necessário)
# Filtra os atributos do tipo 'object'
categorical_cols = X.select_dtypes(include=['object']).columns
print(categorical_cols)

#Transformar atributos categóricos em binários com a função get_dummies()
X_encoded = X.copy()
for col in categorical_cols:
    dummies = pd.get_dummies(X_encoded[col], prefix=col)
    X_encoded = pd.concat([X_encoded, dummies], axis=1)
    X_encoded.drop(col, axis=1, inplace=True)

# Normalizar os atributos contínuos com MinMaxScaler()
mms = MinMaxScaler()

mms.fit(X_encoded)
X_transformed = mms.transform(X_encoded)
X_transformed = pd.DataFrame(X_transformed, columns=X_encoded.columns)
X_transformed.head()

Aplicação do algoritmo K-means no conjunto selecionado com base no número de classes contidas no Target.

In [ ]:
resultados_possiveis = Y['Target'].unique()
k = len(resultados_possiveis)
km = KMeans(n_clusters=k, random_state=42)
km = km.fit(X_transformed)
labels = Y.values.ravel() 
labels_pred = km.labels_
labels, labels_pred

Aplicando os indices de validação ARI (Adjusted Rand Index) e AMI (Adjusted Mutual Information).

In [ ]:
# Calcular ARI e AMI
ari = adjusted_rand_score(labels, labels_pred)
ami = adjusted_mutual_info_score(labels, labels_pred)
print(f'ARI: {ari:.4f}, AMI: {ami:.4f}')

Definindo um intervalo de valores para número de grupos para visualizar a variação do índice no gráfico.

In [ ]:
intervalo = (2,20)

Aplicando o algoritmo k-means no conjunto de dados para todos os valores do intervalo definido. Coletando os valores do índice soma quadrática das distâncias (inertia_) e do índice de validação silhueta (silhouette_score) para cada número de clusters.

In [ ]:
K = range(*intervalo)

Sum_of_squared_distances = []
Silhouette_values = []
for clusters in K:
    km = KMeans(n_clusters=clusters, random_state=42)
    km = km.fit(X_transformed)
    Sum_of_squared_distances.append(km.inertia_)
    Silhouette_values.append(silhouette_score(X_transformed, km.labels_))

Plotando o gráfico dos índices erro quadrático encontrados para cada número de grupos e determinando qual é o melhor de acordo com o método do cotovelo (utilizando o **KElbowVisualizer** da biblioteca **yellowbrick.cluster**).

In [ ]:
model = KMeans(random_state=42)
visualizer = KElbowVisualizer(model, k=intervalo, metric='distortion', timings=False)
visualizer.fit(X_transformed)

plt.plot(K, Sum_of_squared_distances, 'rx-')
plt.xlabel('k')
plt.ylabel('Soma dos quadrados das distâncias')
plt.title('Método do Cotovelo para encontrar melhor valor de k')
plt.show()

Plotando o gráfico dos índices de silhueta encontrados para cada número de grupos e determinando qual é o melhor de acordo com o método do cotovelo (utilizando o **KElbowVisualizer** da biblioteca **yellowbrick.cluster**).

In [ ]:
model = KMeans(random_state=42)
visualizer = KElbowVisualizer(model, k=intervalo, metric='silhouette', timings=False)
visualizer.fit(X_transformed)

plt.plot(K, Silhouette_values, 'rx-')
plt.xlabel('k')
plt.ylabel('Índices de silhueta')
plt.title('Método do Cotovelo para encontrar melhor valor de k')
plt.show()